In [50]:
import numpy as np
from matplotlib import pyplot as plt
import pickle
import glob
import os
import cv2
import torch
from tqdm import tqdm

In [51]:
out_dir = 'out/'
directories = {
    'w_vae': '/home/victor/PycharmProjects/survey_wsl_histology/out/resnet18_vae0.0001/average/',
    'wo_vae': '/home/victor/PycharmProjects/survey_wsl_histology/out/resnet18_vae0/average/'
}

In [52]:
def get_overlays_and_dice(path):
    paths = glob.glob(path + '*.pkl')
    
    overlays = None
    dices = None
    
    objects = {}
    for p in paths:
        with open(p, 'rb') as f:
            pickle_obj = pickle.load(f)
            base = os.path.basename(p)
            key = os.path.splitext(base)[0]
            objects[key] = pickle_obj
                
    return objects

In [53]:
results = {method: get_overlays_and_dice(directory) for (method, directory) in zip(directories.keys(), directories.values())}

In [56]:
with open('npy_results/overlay_gt/input.pkl', 'rb') as f:
    img_input = pickle.load(f)
    img_input = [item.squeeze().transpose(1, 2, 0) for item in img_input]
with open('npy_results/overlay_gt/gt_overlays.pkl', 'rb') as f:
    gt_overlay = pickle.load(f)
    gt_overlay = [item.transpose(1, 2, 0) for item in gt_overlay]

In [57]:
def normalize(x):
    x_min = x.min()
    x_max = x.max()
    return (x - x_min) / (x_max - x_min)

In [68]:
fig_dir = 'results_vae/{}'.format(results.keys())
os.makedirs(fig_dir, exist_ok=True)
for i in tqdm(range(len(img_input)), ncols=80, desc='Saving figures'):
    fig, axs = plt.subplots(1, 4, sharey='row', figsize=(20, 10))
    (ax0, ax1, ax2, ax3) = axs
        
    ax0.imshow(img_input[i])
    ax0.set_title('Input')
    
    ax1.imshow(gt_overlay[i])
    ax1.set_title('GT overlay')
    
    cams_vae, dice_vae, label_vae = results['w_vae']['cams'][i], results['w_vae']['dice_per_image'][i], results['w_vae']['labels'][i]

    cams, dice, label = results['wo_vae']['cams'][i], results['wo_vae']['dice_per_image'][i], results['wo_vae']['labels'][i]

    cam_vae = cams_vae[label_vae]
    cam = cams[label]
    stack = np.stack([cam_vae, cam], axis=0)
    norm_stack = normalize(stack)
    
    ax2.imshow(img_input[i])
    ax2.imshow(255 * norm_stack[0], alpha=0.6, cmap='jet')
    ax2.set_title('w_vae')

    ax3.imshow(img_input[i])
    ax3.imshow(255 * norm_stack[1], alpha=0.6, cmap='jet')
    ax3.set_title('wo_vae')
    
    f_name = 'results_{}.png'.format(i)
    f_path = os.path.join(fig_dir, f_name)
    plt.savefig(f_path, dpi=150)
    plt.close(fig)

Saving figures: 100%|███████████████████████████| 80/80 [02:03<00:00,  1.54s/it]
